In [1]:
from z3 import *
import itertools
from pprint import *


s = Solver()
cells=[[Int('cell_%d_%d' % (r, c)) for c in range(9)] for r in range(9)]

# game = """
# 8 . .   . . .   9 . .
# 7 . .   9 . .   . 3 .
# . . .   . 3 .   1 . 7
# . . .   . . .   . . .
# 4 . 3   1 . .   2 5 .
# . . 6   . . .   . . .
# . . .   . . 7   . . 3
# . 9 7   5 8 .   . . .
# . . .   . . .   4 . .
# """

# game = """
# 8 5 . |. . 2 |4 . . 
# 7 2 . |. . . |. . 9 
# . . 4 |. . . |. . . 
# ------+------+------
# . . . |1 . 7 |. . 2 
# 3 . 5 |. . . |9 . . 
# . 4 . |. . . |. . . 
# ------+------+------
# . . . |. 8 . |. 7 . 
# . 1 7 |. . . |. . . 
# . . . |. 3 6 |. 4 . 
# """

game = """
. . 5 |3 . . |. . . 
8 . . |. . . |. 2 . 
. 7 . |. 1 . |5 . . 
------+------+------
4 . . |. . 5 |3 . . 
. 1 . |. 7 . |. . 6 
. . 3 |2 . . |. 8 . 
------+------+------
. 6 . |5 . . |. . 9 
. . 4 |. . . |. 3 . 
. . . |. . 9 |7 . . 
"""

# game = ".5.1.6......2....69.6..5..8.2.....7...3..........4.5...7..5...919.3..8...3..8...."
# game = ".521.6......2....6..6..5..8.2.....7...3..........4.5...7..5...9.9.3..8...3..8...."
# game = [game[9*i:9*i+9] for i in range(9)]

game = game.replace("|", "").replace("-", "").replace("+", "")
game = [row.split() for row in game.split("\n") if row != '']
pprint(game)

for i in range(9):
    for j in range(9):
        c = game[i][j]
        if c != '.':
            s.add(cells[i][j] == int(c))

# Add cell constraints
for y in range(9):
    for x in range(9):
        s.add(And(1 <= cells[x][y], cells[x][y] <= 9))

# Create row constraints
for row in cells:
     s.add(Distinct(row))

# Create column constraints        
cols = [[row[i] for row in cells] for i in range(9)]
for col in cols:
     s.add(Distinct(col))

# Create subsquare constraints
for i in range(0,9,3):
    for j in range(0,9,3):
        s.add(Distinct([cells[x][y] for x, y in itertools.product(range(i, i+3), range(j, j+3))]))
#         print(list(itertools.product(range(i, i+3), range(j, j+3))))


def add_constraints(m):
    constraints = []
    for i in range(9):
        for j in range(9):
            c = game[i][j]
            if c == '.':
                constraints.append(cells[i][j] != m.evaluate(cells[i][j]))
    return constraints

count = 0 
while s.check() == sat:
    m = s.model()
    count+=1
    print("count:", count)

    solved = [['.' for i in range(9)] for j in range(9)]
    for i in range(9):
        for j in range(9):
            a = m.evaluate(cells[i][j])
            solved[i][j] = a
    pprint(solved)
    s.add(Or(add_constraints(m)))

[['.', '.', '5', '3', '.', '.', '.', '.', '.'],
 ['8', '.', '.', '.', '.', '.', '.', '2', '.'],
 ['.', '7', '.', '.', '1', '.', '5', '.', '.'],
 ['4', '.', '.', '.', '.', '5', '3', '.', '.'],
 ['.', '1', '.', '.', '7', '.', '.', '.', '6'],
 ['.', '.', '3', '2', '.', '.', '.', '8', '.'],
 ['.', '6', '.', '5', '.', '.', '.', '.', '9'],
 ['.', '.', '4', '.', '.', '.', '.', '3', '.'],
 ['.', '.', '.', '.', '.', '9', '7', '.', '.']]
count: 1
[[1, 4, 5, 3, 2, 7, 6, 9, 8],
 [8, 3, 9, 6, 5, 4, 1, 2, 7],
 [6, 7, 2, 9, 1, 8, 5, 4, 3],
 [4, 9, 6, 1, 8, 5, 3, 7, 2],
 [2, 1, 8, 4, 7, 3, 9, 5, 6],
 [7, 5, 3, 2, 9, 6, 4, 8, 1],
 [3, 6, 7, 5, 4, 2, 8, 1, 9],
 [9, 8, 4, 7, 6, 1, 2, 3, 5],
 [5, 2, 1, 8, 3, 9, 7, 6, 4]]
